# Working with columns

## Download and install Spark

In [7]:
#!apt-get update
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
#!tar xf spark-2.3.1-bin-hadoop2.7.tgz
#!pip install -q findspark

## Setup environment

In [8]:
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

## Downloading and preprocessing Chicago's Reported Crime Data

In [9]:
#!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
#!ls -l

In [10]:
#!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv
#!ls -l

In [11]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('../../reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
rc.show(5)

+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+----+--------+------------+------------+----+-----------+-------------+--------------------+
|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|Ward|FBI Code|X Coordinate|Y Coordinate|Year|   Latitude|    Longitude|            Location|
+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+----+--------+------------+------------+----+-----------+-------------+--------------------+
|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|   8|      11|        null|        null|2001|       null|         null|                null|
|    G553545|2001-09-15 02:00:00|     013XX W POLK ST|0460|     

## Working with columns

**Display only the first 5 rows of the column name IUCR **

In [12]:
rc.select("IUCR").show(5)

+----+
|IUCR|
+----+
|1153|
|0460|
|0810|
|0460|
|0110|
+----+
only showing top 5 rows



In [15]:
rc.select(rc.IUCR).show(5)

+----+
|IUCR|
+----+
|1153|
|0460|
|0810|
|0460|
|0110|
+----+
only showing top 5 rows



In [16]:
rc.select(col("IUCR")).show(5)

+----+
|IUCR|
+----+
|1153|
|0460|
|0810|
|0460|
|0110|
+----+
only showing top 5 rows



  **Display only the first 4 rows of the column names Case Number, Date and Arrest**

In [19]:
rc.select(["Case Number", "Date", "Arrest"]).show(4)

+-----------+-------------------+------+
|Case Number|               Date|Arrest|
+-----------+-------------------+------+
|   JA366925|2001-01-01 11:00:00| false|
|    G553545|2001-09-15 02:00:00| false|
|    G666677|2001-11-02 18:30:00| false|
|    G749215|2001-12-15 02:00:00| false|
+-----------+-------------------+------+
only showing top 4 rows



In [22]:
rc.select("Case Number", "Date", "Arrest").show(4)

+-----------+-------------------+------+
|Case Number|               Date|Arrest|
+-----------+-------------------+------+
|   JA366925|2001-01-01 11:00:00| false|
|    G553545|2001-09-15 02:00:00| false|
|    G666677|2001-11-02 18:30:00| false|
|    G749215|2001-12-15 02:00:00| false|
+-----------+-------------------+------+
only showing top 4 rows



** Add a column with name One, with entries all 1s **

In [23]:
from pyspark.sql.functions import lit

In [25]:
rc.withColumn("One", lit(1)).show(5)

+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+----+--------+------------+------------+----+-----------+-------------+--------------------+---+
|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|Ward|FBI Code|X Coordinate|Y Coordinate|Year|   Latitude|    Longitude|            Location|One|
+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+----+--------+------------+------------+----+-----------+-------------+--------------------+---+
|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|   8|      11|        null|        null|2001|       null|         null|                null|  1|
|    G553545|2001-09-15 02:00:00|     013XX W PO

In [26]:
rc.withColumn("One", lit(1)).select("One").show(5)

+---+
|One|
+---+
|  1|
|  1|
|  1|
|  1|
|  1|
+---+
only showing top 5 rows



** Remove the column IUCR **

In [28]:
rc = rc.drop("IUCR")

rc.show(5)

+-----------+-------------------+--------------------+------------------+--------------------+--------------------+------+--------+----+----+--------+------------+------------+----+-----------+-------------+--------------------+
|Case Number|               Date|               Block|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|Ward|FBI Code|X Coordinate|Y Coordinate|Year|   Latitude|    Longitude|            Location|
+-----------+-------------------+--------------------+------------------+--------------------+--------------------+------+--------+----+----+--------+------------+------------+----+-----------+-------------+--------------------+
|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|   8|      11|        null|        null|2001|       null|         null|                null|
|    G553545|2001-09-15 02:00:00|     013XX W POLK ST|           BATTERY|           

In [29]:
rc.columns

['Case Number',
 'Date',
 'Block',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'Ward',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Latitude',
 'Longitude',
 'Location']